<a href="https://colab.research.google.com/github/LuckyBoy587/Notes-Summarizer/blob/master/notebooks/notes_summarizer_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes Summarizer (Colab)

This notebook is intentionally minimal: it only installs dependencies, clones the repository, and calls the project's `main.py`. No helper functions or UI code are embedded here.

Run the cells in order. After cloning, `main.py` will be invoked (it expects a PDF path argument; run it as shown or upload a file and adapt the command below).

In [ ]:
# Install dependencies and clone the repository (quiet installs)
!pip install -q -r requirements.txt || true
!pip install -q PyMuPDF || true
!git clone https://github.com/LuckyBoy587/Notes-Summarizer.git || true
%cd Notes-Summarizer

# Download NLTK punkt data used by text_processing.split_into_topics
import nltk
nltk.download('punkt')

# Use Colab's file upload UI, call main.summarize_pdf on each uploaded file, then download results
from google.colab import files
import main
uploaded = files.upload()
for fname in uploaded.keys():
    print('Processing', fname)
    out = main.summarize_pdf(fname, paraphrase=True)
    print('Generated', out)
    try:
        files.download(out)
    except Exception as e:
        print('Download failed:', e)